In [1]:
import geopandas as gpd
import pandas as pd
import osmnx as ox
import momepy

# Convert shapefile to graph
hghway = gpd.read_file("./data/oregon_highways.shp")
graph = momepy.gdf_to_nx(hghway, approach="primal", directed=True)

In [4]:
urlBike = "https://maps.amsterdam.nl/open_geodata/geojson_latlng.php?KAARTLAAG=FIETSNETTEN&THEMA=fietsnetten"
bike = gpd.read_file(urlBike)
amBike = bike.explode()
#explode multi line to single line .explode
graph = momepy.gdf_to_nx(amBike, approach="primal", directed=True)

/var/folders/6h/mjkfjbdx5j33s6js5qfxw5f40000gn/T/ipykernel_7379/3853366709.py:3: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  amBike = bike.explode()
/Users/kelseymccravy/.gds/lib/python3.12/site-packages/momepy/utils.py:252: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_network[length] = gdf_network.geometry.length


In [5]:
# Convert graph to gdf
nodes, edges = momepy.nx_to_gdf(graph)

In [6]:
# Provide an x and y for OSMnx
nodes['x'] = nodes['geometry'].x
nodes['y'] = nodes['geometry'].y

In [7]:
# Provide a u and v for OSMnx
edges['u'] = edges['node_start']
edges['v'] = edges['node_end']

In [8]:
# Make all roads "two-way"
opposite_direction = edges.copy()
opposite_direction = opposite_direction.rename(columns={"u": "v", "v": "u"})
directed_edges = pd.concat([edges, opposite_direction], ignore_index=True)

In [9]:
edges = directed_edges.set_index(['u', 'v'])

In [10]:
# Add a key index
doubles = edges[edges.index.duplicated()].copy()
singles = edges[~edges.index.duplicated()].copy()

singles['key'] = 0
doubles['key'] = 1

edges = pd.concat([singles, doubles])

In [11]:
# Make the multi-index
edges = edges.reset_index().set_index(['u', 'v', 'key'])

In [12]:
# Rename some columns
edges = edges[['HWYNAME','mm_len', 'geometry']]
edges = edges.rename(columns={"HWYNAME": "name", "mm_len": "length"})

KeyError: "['HWYNAME'] not in index"

In [ ]:
# Convery back to graph
gdf = ox.graph_from_gdfs(nodes, edges)

In [ ]:
# Save and load
ox.save_graphml(gdf, './data/graph.graphml')
graph = ox.load_graphml('./data/graph.graphml')

In [ ]:
# Plot
fig, ax = ox.plot_graph(graph, bgcolor='white', node_color='black', edge_color='grey', node_size=5)